In [2]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd

# Load the environment variables from the .env file
load_dotenv()

# Define the parameters for the API requests
CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")
REDIRECT_URI = os.environ.get("REDIRECT_URI")
RESTAURANT_ID = os.environ.get("RESTAURANT_ID")
ACCESS_TOKEN = os.environ.get("ACCESS_TOKEN")
REFRESH_TOKEN = os.environ.get("REFRESH_TOKEN")

# Set API details
# Use the access token to interact with the Dinlr API
base_url = "https://api.dinlr.com/v1"
aheaders = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

rheaders = {
    "Authorization": f"Bearer {REFRESH_TOKEN}"
}
lTUG_ID = 'a7e56eeb-98db-402d-bea1-1ee35d54a4fd'
lEVENT_ID = '6bf38c18-a852-439b-aa54-007547f1bb6a'

# Send the GET request to the locations endpoint
# response = requests.get(f"https://api.dinlr.com/v1/{RESTAURANT_ID}/onlineorder/locations", headers=aheaders)
# data = response.json()

/tmp/ipykernel_2081/2945317536.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Defining the functions to get order and order details.

In [8]:
# Function to get all orders
# Function to get all orders
def get_all_orders(location_id):
    page = 1
    orders = []
    
    try:
        while True:
            url = f"{base_url}/{RESTAURANT_ID}/onlineorder/orders?location_id={location_id}&page={page}"
            response = requests.get(url, headers=aheaders)
            data = response.json()["data"]
            
            if not data:
                break
            
            orders.extend(data)
            page += 1
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    
    return orders

# Function to get order details
def get_order_details(order_id):
    url = f"{base_url}/{RESTAURANT_ID}/onlineorder/orders/{order_id}"
    response = requests.get(url, headers=aheaders)
    order_details = response.json()["data"]
    return order_details

In [9]:
# Fetch all orders and their details
TUG_orders = get_all_orders(lTUG_ID)
EVENT_orders = get_all_orders(lEVENT_ID)

TUG_all_order_details = [get_order_details(order["id"]) for order in TUG_orders]
EVENT_all_order_details = [get_order_details(order["id"]) for order in EVENT_orders]

# Convert to DataFrame
TUG_df_orders = pd.DataFrame(TUG_all_order_details)
EVENT_df_orders = pd.DataFrame(EVENT_all_order_details)

In [10]:
TUG_df_orders.count()

id                         6394
customer                    100
order_no                   6394
order_ticket                  1
dining_option              6394
dining_option_name         6394
pax                        6394
subtotal                   6394
total                      6394
rounding                   6394
paid                       6394
status                     6394
financial_status           6394
kitchen_status               10
expedite_status              10
notes                        14
void_reason                  54
updated_at                 6394
created_at                 6394
objects                    6394
items                      6394
discounts                  6394
charges                    6394
taxes                      6394
manufacturer_discounts     6394
loyalty_program_rewards    6394
vouchers                   6394
payments                   6394
refunds                    6394
dtype: int64

In [11]:
len(TUG_orders)

6394

In [44]:
parsed = json.loads(TUG_df_orders.to_json(orient="records"))
with open("TUG_orders.json", "w") as json_file:
    json.dump(parsed, json_file, indent=4)

parsed = json.loads(EVENT_df_orders.to_json(orient="records"))
with open("EVENT_orders.json", "w") as json_file:
    json.dump(parsed, json_file, indent=4)

In [37]:
TUG_df_orders_items = pd.json_normalize(TUG_df_orders['items'].explode())

Save to parquet / json

In [ ]:
# # Save to Parquet with GZIP compression
# parquet_file = '/mnt/data/orders.parquet.gzip'
# df_orders.to_parquet(parquet_file, compression='gzip')